In [2]:
#imports
import os
import pathlib

import numpy as np
import torch
from torch.utils.data import DataLoader
from torchvision.models.detection.transform import GeneralizedRCNNTransform

from pytorch_faster_rcnn.datasets import ObjectDetectionDatasetSingle, ObjectDetectionDataSet
from pytorch_faster_rcnn.faster_RCNN import get_faster_rcnn_resnet
from pytorch_faster_rcnn.transformations import ComposeDouble
from pytorch_faster_rcnn.transformations import ComposeSingle
from pytorch_faster_rcnn.transformations import FunctionWrapperDouble
from pytorch_faster_rcnn.transformations import FunctionWrapperSingle
from pytorch_faster_rcnn.transformations import apply_nms, apply_score_threshold
from pytorch_faster_rcnn.transformations import normalize_01
from pytorch_faster_rcnn.utils import get_filenames_of_path, collate_single, save_json
from pytorch_faster_rcnn.visual import DatasetViewer
from pytorch_faster_rcnn.visual import DatasetViewerSingle
from pytorch_faster_rcnn.backbone_resnet import ResNetBackbones

In [3]:
#initiate parameters for inference
params = {'INPUT_DIR': 'pytorch_faster_rcnn/data/shelves/test',  # input files for which to generate prediction
          'PREDICTIONS_PATH': 'pytorch_faster_rcnn/data/shelves/predictions',  #predictions save directory
          'MODEL_DIR': 'experiment1',  #directory to load trained models from
          'VERSION': 'version_6', #specific version to use for inference
          }

In [4]:
#load input files for inference/prediction (unlabeled grocery images)
inputs = get_filenames_of_path(pathlib.Path(params['INPUT_DIR']))
inputs.sort()

In [5]:
#transformations to apply to input data
transforms = ComposeSingle([
    FunctionWrapperSingle(np.moveaxis, source=-1, destination=0),
    FunctionWrapperSingle(normalize_01)
    ])

In [6]:
# create dataset (single because it has no targets)
dataset = ObjectDetectionDatasetSingle(inputs=inputs,
                                       transform=transforms,
                                       use_cache=False,
                                       )

In [7]:
# create dataloader
dataloader_prediction = DataLoader(dataset=dataset,
                                   batch_size=1,
                                   shuffle=False,
                                   num_workers=0,
                                   collate_fn=collate_single)

In [11]:
#load model from version indicated in params
checkpoint_path = str(os.getcwd()) +"/" + params['MODEL_DIR'] + "/" + params['VERSION'] + "/checkpoints"
for file in os.listdir(checkpoint_path):
    checkpoint_path += str("/" + os.fsdecode(file))
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model_state_dict = checkpoint['hyper_parameters']['model'].state_dict()

In [12]:
#initiate model with same parameters as version indicated in params (same backbone, anchor sizes, aspect_ratios, etc.)
model = get_faster_rcnn_resnet(num_classes=2,
                               backbone_name= ResNetBackbones.RESNET50,  
                               anchor_size=((128, 256, 512),),
                               aspect_ratios=((0.5, 1.0, 2.0),),
                               fpn=False,
                               min_size=1024,
                               max_size=1025
                               )

In [13]:
#load weights onto model (!only works if params specified above match the one used in the trained version that is loaded)
model.load_state_dict(model_state_dict)

RuntimeError: Error(s) in loading state_dict for FasterRCNN:
	Unexpected key(s) in state_dict: "backbone.5.4.conv1.weight", "backbone.5.4.bn1.weight", "backbone.5.4.bn1.bias", "backbone.5.4.bn1.running_mean", "backbone.5.4.bn1.running_var", "backbone.5.4.bn1.num_batches_tracked", "backbone.5.4.conv2.weight", "backbone.5.4.bn2.weight", "backbone.5.4.bn2.bias", "backbone.5.4.bn2.running_mean", "backbone.5.4.bn2.running_var", "backbone.5.4.bn2.num_batches_tracked", "backbone.5.4.conv3.weight", "backbone.5.4.bn3.weight", "backbone.5.4.bn3.bias", "backbone.5.4.bn3.running_mean", "backbone.5.4.bn3.running_var", "backbone.5.4.bn3.num_batches_tracked", "backbone.5.5.conv1.weight", "backbone.5.5.bn1.weight", "backbone.5.5.bn1.bias", "backbone.5.5.bn1.running_mean", "backbone.5.5.bn1.running_var", "backbone.5.5.bn1.num_batches_tracked", "backbone.5.5.conv2.weight", "backbone.5.5.bn2.weight", "backbone.5.5.bn2.bias", "backbone.5.5.bn2.running_mean", "backbone.5.5.bn2.running_var", "backbone.5.5.b

In [27]:
# inference (cpu) --> makes the predictions and saves then in predictions_path as specified in params
model.eval()
for sample in dataloader_prediction:
    x, x_name = sample
    with torch.no_grad():
        pred = model(x)
        pred = {key: value.numpy() for key, value in pred[0].items()}
        name = pathlib.Path(x_name[0])
        save_dir = pathlib.Path(os.getcwd()) / params['PREDICTIONS_PATH']
        save_dir.mkdir(parents=True, exist_ok=True)
        pred_list = {key: value.tolist() for key, value in pred.items()}  # numpy arrays are not serializable -> .tolist()
        save_json(pred_list, path=save_dir / name.with_suffix('.json'))

KeyboardInterrupt: 

In [28]:
# get prediction files (json) & sort them
predictions = get_filenames_of_path(pathlib.Path(os.getcwd()) / params['PREDICTIONS_PATH'])
predictions.sort()

In [37]:
# set IoU threshold & score threshold
iou_threshold = 0.25
score_threshold = 0.5

# set transformations applied to dataset with predictions (can experiment with nms and score threshold)
transforms_prediction = ComposeDouble([
    FunctionWrapperDouble(np.moveaxis, source=-1, destination=0),
    FunctionWrapperDouble(normalize_01),
    FunctionWrapperDouble(apply_nms, input=False, target=True, iou_threshold=iou_threshold),
    FunctionWrapperDouble(apply_score_threshold, input=False, target=True, score_threshold=score_threshold)
])

#create dataset with predictions
dataset_prediction = ObjectDetectionDataSet(inputs=inputs,
                                            targets=predictions,
                                            transform=transforms_prediction,
                                            use_cache=False)

In [31]:
# mapping
color_mapping = {
    1: 'red',
}

# visualize predictions
datasetviewer_prediction = DatasetViewer(dataset_prediction, color_mapping)
datasetviewer_prediction.napari()
# add text properties gui
datasetviewer_prediction.gui_text_properties(datasetviewer_prediction.shape_layer)